In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/bit_conference/

/content/gdrive/.shortcut-targets-by-id/1YDrmXvwQeDTF3AVegVo_-qlULY2-1-qE/bit_conference


In [ ]:
!pip install datasets

### 이미지 폴더 정리

In [ ]:
import os
import shutil

data_dir = "/content/gdrive/MyDrive/bit_conference/image_gen"
emotions = ["Exciting", "Hopeful", "Romantic", "Heartwarming", "Calm", "Sad", "Stress", "Lonely"]

for emotion in emotions:
    lower_emotion = emotion.lower()
    src_folder = os.path.join(data_dir, lower_emotion)  # 예: 'exciting'
    dst_folder = os.path.join(data_dir, lower_emotion + "_aug")  # 예: 'exciting_aug'
    renamed_folder = os.path.join(data_dir, emotion)  # 예: 'Exciting'

    # 이미지 파일 이동
    if os.path.exists(src_folder) and os.path.exists(dst_folder):
        for file_name in os.listdir(src_folder):
            src_file = os.path.join(src_folder, file_name)
            dst_file = os.path.join(dst_folder, file_name)
            if os.path.isfile(src_file):  # 파일만 이동
                shutil.move(src_file, dst_file)

        # 원본 폴더 삭제 (선택사항)
        os.rmdir(src_folder)

        # dst_folder 이름 변경
        os.rename(dst_folder, renamed_folder)
    else:
        print(f"Skipping {emotion}: One or both folders not found.")

print("Processing complete.")

Processing complete.


In [ ]:
import os
import shutil

# 설정된 경로
base_path = "/content/gdrive/MyDrive/bit_conference/"
source_folder = os.path.join(base_path, "image_gen")
target_folder = os.path.join(base_path, "image_aug")

# 감정 폴더 리스트
EMOTIONS = ["Exciting", "Hopeful", "Romantic", "Heartwarming", "Calm", "Sad", "Stress", "Lonely"]

# 새로운 image_aug 폴더 및 하위 폴더 생성
os.makedirs(target_folder, exist_ok=True)
for emotion in EMOTIONS:
    os.makedirs(os.path.join(target_folder, emotion), exist_ok=True)

# 각 감정 폴더를 순회하며 '_aug'가 포함된 이미지 파일 이동
for emotion in EMOTIONS:
    source_emotion_folder = os.path.join(source_folder, emotion)
    target_emotion_folder = os.path.join(target_folder, emotion)

    if os.path.exists(source_emotion_folder):
        for filename in os.listdir(source_emotion_folder):
            if "_aug" in filename and filename.endswith(".png"):
                src_path = os.path.join(source_emotion_folder, filename)
                dst_path = os.path.join(target_emotion_folder, filename)
                shutil.move(src_path, dst_path)
                print(f"Moved: {src_path} -> {dst_path}")

print("All '_aug' images have been moved successfully.")

In [ ]:
# image_aug의 하위 폴더에서 파일명의 마지막 글자가 '2'인 png 파일 삭제
for emotion in EMOTIONS:
    target_emotion_folder = os.path.join(target_folder, emotion)
    if os.path.exists(target_emotion_folder):
        for filename in os.listdir(target_emotion_folder):
            if filename.endswith("2.png"):
                file_path = os.path.join(target_emotion_folder, filename)
                os.remove(file_path)
                print(f"Deleted: {file_path}")

print("All images ending with '2.png' have been deleted successfully.")

### Efficientnet 파인튜닝

In [ ]:
import os
import torch
import random
from PIL import Image
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from transformers import AutoImageProcessor, AutoModelForImageClassification, TrainingArguments, Trainer

# 데이터셋 경로 및 감정 레이블 정의
DATASET_PATH = "/content/gdrive/MyDrive/bit_conference/image_aug"
EMOTIONS = ["Exciting", "Hopeful", "Romantic", "Heartwarming", "Calm", "Sad", "Stress", "Lonely"]

# 이미지 데이터셋 클래스
def load_dataset():
    dataset = []
    for label, emotion in enumerate(EMOTIONS):
        emotion_path = os.path.join(DATASET_PATH, emotion)
        for img_name in os.listdir(emotion_path):
            img_path = os.path.join(emotion_path, img_name)
            dataset.append((img_path, label))
    random.shuffle(dataset)  # 데이터 순서 섞기
    return dataset

class EmotionDataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img_path, label = self.dataset[idx]
        image = Image.open(img_path).convert("RGB")
        inputs = self.processor(image, return_tensors="pt")
        inputs = {k: v.squeeze(0) for k, v in inputs.items()}
        inputs["labels"] = torch.tensor(int(label), dtype=torch.int64)
        return inputs

# 데이터 불러오기
dataset = load_dataset()
random.shuffle(dataset)
train_size = int(0.8 * len(dataset))
train_data, val_data = dataset[:train_size], dataset[train_size:]

# 이미지 프로세서 로드
processor = AutoImageProcessor.from_pretrained("google/efficientnet-b0")
train_dataset = EmotionDataset(train_data, processor)
val_dataset = EmotionDataset(val_data, processor)

# 모델 로드 및 수정
model = AutoModelForImageClassification.from_pretrained(
    "google/efficientnet-b0",
    num_labels=len(EMOTIONS),
    ignore_mismatched_sizes=True
)

# ResNet Backbone Freeze
for param in model.efficientnet.parameters():
    param.requires_grad = False

# Fully Connected Layer만 학습
for param in model.classifier.parameters():
    param.requires_grad = True

# 훈련 설정
training_args = TrainingArguments(
    output_dir="/content/gdrive/MyDrive/bit_conference/efficientnet_emotion_model",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=5,
    logging_dir="/content/gdrive/MyDrive/bit_conference/logs",
    logging_steps=10,
    save_total_limit=2,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# 모델 학습
trainer.train()

# 모델 저장
model.save_pretrained("/content/gdrive/MyDrive/bit_conference/efficientnet_emotion_model")
processor.save_pretrained("/content/gdrive/MyDrive/bit_conference/efficientnet_emotion_model")


Some weights of EfficientNetForImageClassification were not initialized from the model checkpoint at google/efficientnet-b0 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 1280]) in the checkpoint and torch.Size([8, 1280]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([8]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.987400,2.024227
2,2.014400,2.003453
3,1.918700,1.946142
4,1.857900,1.930461


['/content/gdrive/MyDrive/bit_conference/efficientnet_emotion_model/preprocessor_config.json']

### 결과

In [ ]:
import os
import torch
import pandas as pd
from tqdm import tqdm
from PIL import Image
from transformers import AutoImageProcessor, AutoModelForImageClassification

# 모델 및 프로세서 로드 (반정밀도 연산 적용)
model_path = "/content/gdrive/MyDrive/bit_conference/efficientnet_emotion_model"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델을 반정밀도로 로드
model = AutoModelForImageClassification.from_pretrained(model_path).to(device).half()
processor = AutoImageProcessor.from_pretrained(model_path)
model.eval()

# 감정 레이블 정의
EMOTIONS = ["Exciting", "Hopeful", "Romantic", "Heartwarming", "Calm", "Sad", "Stress", "Lonely"]

# 이미지 폴더 경로
image_gen_path = "/content/gdrive/MyDrive/bit_conference/image_gen"

# 데이터 저장을 위한 딕셔너리
results = {emotion: [] for emotion in EMOTIONS}

# 이미지 예측 함수
def predict(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(image, return_tensors="pt")

    # GPU로 이동 및 반정밀도 적용 (반드시 tensor에만 적용)
    inputs = {k: v.to(device).half() for k, v in inputs.items()}

    with torch.no_grad():
        logits = model(**inputs).logits

    probs = torch.nn.functional.softmax(logits, dim=-1).squeeze()
    predicted_label = probs.argmax().item()

    return predicted_label, probs.cpu().numpy()

# 모든 이미지에 대해 예측 수행 및 결과 저장 (tqdm 추가)
for emotion in EMOTIONS:
    folder_path = os.path.join(image_gen_path, emotion)
    if os.path.exists(folder_path):
        image_files = [f for f in os.listdir(folder_path) if f.endswith(".png")]
        for image_file in tqdm(image_files, desc=f"Processing {emotion}"):
            image_path = os.path.join(folder_path, image_file)
            pred_label, probs = predict(image_path)

            results[emotion].append({
                "image": image_file,
                "predicted_label": EMOTIONS[pred_label],
                **{EMOTIONS[i]: probs[i] for i in range(len(EMOTIONS))}
            })

            # 메모리 절약을 위해 캐시 정리
            del pred_label, probs
            torch.cuda.empty_cache()

# 데이터프레임 생성 및 출력
dfs = {emotion: pd.DataFrame(results[emotion]) for emotion in EMOTIONS}


Processing Lonely: 100%|██████████| 463/463 [00:35<00:00, 12.98it/s]


In [ ]:
for emotion in EMOTIONS:
  print(emotion)
  print(dfs[emotion]['predicted_label'].value_counts())

Exciting
predicted_label
Exciting        126
Heartwarming    100
Hopeful          53
Stress           42
Romantic         34
Lonely           32
Calm             24
Sad              12
Name: count, dtype: int64
Hopeful
predicted_label
Heartwarming    78
Hopeful         71
Exciting        65
Romantic        54
Stress          50
Lonely          31
Calm            29
Sad             19
Name: count, dtype: int64
Romantic
predicted_label
Romantic        153
Heartwarming     70
Lonely           64
Sad              47
Stress           42
Hopeful          39
Exciting         29
Calm             13
Name: count, dtype: int64
Heartwarming
predicted_label
Heartwarming    225
Exciting         75
Stress           47
Hopeful          41
Romantic         37
Lonely           22
Sad              17
Calm             16
Name: count, dtype: int64
Calm
predicted_label
Calm            83
Romantic        77
Hopeful         66
Lonely          53
Stress          47
Sad             44
Exciting        40
Heartwa